In [4]:
pip install h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 2.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install torch

  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB ? eta 0:00:000:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 1.4 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB ? eta 0:00:000:00:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 372.0 kB/s eta 0:00:0000:0100:01�╸ 168.4/168.4 MB 5.7 MB/s eta 0:00:01�╸ 168.4/168.4 MB 5.7 MB/s eta 0:00:01�╸ 168.4/168.4 MB 5.7 MB/s eta 0:00:01
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
pip install torchani

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 3.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.5 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 927.7 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyanitools as pya
import numpy as np
from torch import nn

In [2]:
from pyanitools import anidataloader

In [3]:
import torchani
import torch 

/home/jeffy011/CHEM277B/msse/lib/python3.10/site-packages/torchani/aev.py:16: UserWarning: cuaev not installed
  warnings.warn("cuaev not installed")
/home/jeffy011/CHEM277B/msse/lib/python3.10/site-packages/torchani/__init__.py:55: UserWarning: Dependency not satisfied, torchani.ase will not be available
  warnings.warn("Dependency not satisfied, torchani.ase will not be available")


In [4]:
from torch.optim import SGD, Adam
import torch.nn.functional as F
import random
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.model_selection import KFold

### Understanding the dataset 

In [5]:
# Set the HDF5 file containing the data
data = anidataloader("../ani_gdb_s01.h5")
data_iter = data.__iter__()

mols = next(data_iter)
# Extract the data
P = mols['path']
X = mols['coordinates']
E = mols['energies']
S = mols['species']
sm = mols['smiles']

# Print the data
print("Path:   ", P)
print("  Smiles:      ","".join(sm))
print("  Symbols:     ", S)
print("  Coordinates: ", X.shape)
print("  Energies:    ", E.shape, "\n")

Path:    /gdb11_s01/gdb11_s01-0
  Smiles:       [H]C([H])([H])[H]
  Symbols:      ['C', 'H', 'H', 'H', 'H']
  Coordinates:  (5400, 5, 3)
  Energies:     (5400,) 



In [6]:
data_iter = data.__iter__()
count = 0 # count total number of molecules 
count_conf = 0 # count configuration 
species_set = set() # set to store unique species

for mol in data_iter:
    count += 1 
    count_conf += len(mol['energies'])
    species_set.update(set(mol['species'])) # add unique species to set

print("Total number of molecules:", count)
print("Total number of configurations:", count_conf)
print("Total number of unique species:", len(species_set))
print(species_set)

Total number of molecules: 3
Total number of configurations: 10800
Total number of unique species: 4
{'H', 'N', 'O', 'C'}


As seen above, the dataset has several key features and mainly involves the smiles representation of the molecule showing how the atoms bond with each other. Additionally, the symbols of the atoms are also presented along with the total number of conformations (5400 in this case). In the coordinates variable, the number 5 presents the total number of atoms and the number 3 represents that there are three coordiantes for each atom. Further analysis shows that the file itself has 5 such molecules with over 10000 configurations. 

### Performing additional functions

In [7]:
output = []
Energies = []
# Set the HDF5 file containing the data
data = anidataloader("../ani_gdb_s01.h5")
data_iter = data.__iter__()

for mol in data_iter: 
    # Extract the data
    P = mols['path']
    X = mols['coordinates']
    E = mols['energies']
    Energies.append(E)
    S = mols['species']
    sm = mols['smiles']

    #constants used when defining AEV 
    Rcr = 5.2
    EtaR = torch.tensor([16], dtype=torch.float)
    ShfR = torch.tensor([0.900000,1.168750,1.437500,1.706250,1.975000,2.243750,2.51250,2.781250,3.050000,3.318750,3.587500,3.856250,4.125000,4.39375,4.662500,4.931250])
    Rca = 3.5
    EtaA = torch.tensor([8], dtype=torch.float)
    ShfA = torch.tensor([0.900000,1.550000,2.200000,2.850000], dtype=torch.float)
    ShfZ = torch.tensor([0.19634954,0.58904862,0.9817477,1.3744468,1.7671459,2.1598449,2.552544,2.945243]) 
    Zeta = torch.tensor([32], dtype=torch.float)
    species_order = ['H', 'C', 'N', 'O']
    num_species = len(species_order)

    #should write this in the correct order to run 
    aev_computer = torchani.AEVComputer(Rcr, Rca, EtaR, ShfR, EtaA,Zeta, ShfA, ShfZ, num_species)

    mapping = {"H":0, "C":1, "N":2, "O":3}
    species = np.array([mapping[atom] for atom in S])
    print(S)
    print(species)
    species = np.tile(species, (X.shape[0], 1))
    species = torch.tensor(species)
    X = torch.tensor(X)

    aev_output = aev_computer((species, X)) #Species: (N,A) and coordinate:(N,A,3) and output: (N,A, #num_AEV)  
    aev_output[1].shape

    aev_output

    output.append(aev_output)
    
print(aev_output[0])

['C', 'H', 'H', 'H', 'H']
[1 0 0 0 0]
['C', 'H', 'H', 'H', 'H']
[1 0 0 0 0]
['C', 'H', 'H', 'H', 'H']
[1 0 0 0 0]
tensor([[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        ...,
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0]])


Once the basics of the data set has been analyzed, much thought was given into represenation of the data set in order to create a system that accurately depicts the coformation of the atoms while also taking into account the various details such as transaltional, rotational and angular motions. This is done using the AEV calculator which acts as a representation of the atoms. This is done for each of the atoms in each of the molecules and saved for further analysis.

### Creating an initial neural network 

This section is still under contruction. I have an idea of what I need to do and what I need to change but I have not been able to get a functioning code yet. 

In [11]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' % (f.__name__,  te-ts))
        return result
    return wrap

def create_chunks(complete_list, chunk_size=None, num_chunks=None):
    '''
    Cut a list into multiple chunks, each having chunk_size (the last chunk might be less than chunk_size) or having a total of num_chunk chunks
    '''
    chunks = []
    if num_chunks is None:
        num_chunks = math.ceil(len(complete_list) / chunk_size)
    elif chunk_size is None:
        chunk_size = math.ceil(len(complete_list) / num_chunks)
    for i in range(num_chunks):
        chunks.append(complete_list[i * chunk_size: (i + 1) * chunk_size])
    return chunks

class Trainer():
    def __init__(self, model, optimizer_type, learning_rate, epoch, batch_size, input_transform=lambda x: x):
        """ The class for training the model
        model: nn.Module
            A pytorch model
        optimizer_type: 'adam' or 'sgd'
        learning_rate: float
        epoch: int
        batch_size: int
        input_transform: func
            transforming input. Can do reshape here
        """
        self.model = model
        if optimizer_type == "sgd":
            self.optimizer = SGD(model.parameters(), learning_rate,momentum=0.9)
        elif optimizer_type == "adam":
            self.optimizer = Adam(model.parameters(), learning_rate)
            
        self.epoch = epoch
        self.batch_size = batch_size
        self.input_transform = input_transform


    @timing
    def train(self, inputs, outputs, val_inputs, val_outputs,early_stop=False,l2=False,silent=False):
        """ train self.model with specified arguments
        inputs: np.array, The shape of input_transform(input) should be (ndata,nfeatures)
        outputs: np.array shape (ndata,)
        val_nputs: np.array, The shape of input_transform(val_input) should be (ndata,nfeatures)
        val_outputs: np.array shape (ndata,)
        early_stop: bool
        l2: bool
        silent: bool. Controls whether or not to print the train and val error during training
        
        @return
        a dictionary of arrays with train and val losses and accuracies
        """
        inputs = self.input_transform(inputs)
        val_inputs = self.input_transform(val_inputs)
        ### convert data to tensor of correct shape and type here ###
        inputs = torch.tensor(inputs, dtype=torch.float)
        outputs = torch.tensor(outputs, dtype=torch.int64)
        losses = []
        accuracies = []
        val_losses = []
        val_accuracies = []
        weights = self.model.state_dict()
        lowest_val_loss = np.inf
        
        for n_epoch in tqdm(range(self.epoch), leave=False):
            self.model.train()
            batch_indices = list(range(inputs.shape[0]))
            random.shuffle(batch_indices)
            batch_indices = create_chunks(batch_indices, chunk_size=self.batch_size)
            epoch_loss = 0
            epoch_acc = 0
            for batch in batch_indices:
                batch_importance = len(batch) / len(outputs)
                batch_input = inputs[batch]
                batch_output = outputs[batch]
                ### make prediction and compute loss with loss function of your choice on this batch ###
                batch_predictions = self.model(batch_input)
                #will need to adjust to account for a continous model 
#                 loss = nn.CrossEntropyLoss()(batch_predictions, batch_output)
                if l2:
                    ### Compute the loss with L2 regularization ###
                    l2_lambda = 0.00001
                    l2_norm = sum(p.pow(2.0).sum() for p in self.model.parameters())
                    loss += l2_lambda * l2_norm
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                ### Compute epoch_loss and epoch_acc
                epoch_loss += loss.detach().item() * batch_importance
                pred = torch.argmax(batch_predictions, axis=-1)
                #will need to adjust to account for continuous model
#                 acc = torch.sum(pred == batch_output).item() / len(batch_predictions)
                epoch_acc += acc * batch_importance
            val_loss, val_acc = self.evaluate(val_inputs, val_outputs, print_acc=False)
            if n_epoch % 10 ==0 and not silent: 
                print("Epoch %d/%d - Loss: %.3f - Acc: %.3f" % (n_epoch + 1, self.epoch, epoch_loss, epoch_acc))
                print("              Val_loss: %.3f - Val_acc: %.3f" % (val_loss, val_acc))
            losses.append(epoch_loss)
            accuracies.append(epoch_acc)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)
            if early_stop:
                if val_loss < lowest_val_loss:
                    lowest_val_loss = val_loss
                    weights = self.model.state_dict()

        if early_stop:
            self.model.load_state_dict(weights)    

        return {"losses": losses, "accuracies": accuracies, "val_losses": val_losses, "val_accuracies": val_accuracies}
        
    def evaluate(self, inputs, outputs, print_acc=True):
        """ evaluate model on provided input and output
        inputs: np.array, The shape of input_transform(input) should be (ndata,nfeatures)
        outputs: np.array shape (ndata,)
        print_acc: bool
        
        @return
        losses: float
        acc: float
        """
        inputs = self.input_transform(inputs)
        inputs = torch.tensor(inputs, dtype=torch.float32)
        outputs = torch.tensor(outputs, dtype=torch.int64)
        self.model.eval()
        batch_indices = list(range(inputs.shape[0]))
        batch_indices = create_chunks(batch_indices, chunk_size=self.batch_size)
        acc = 0
        losses = 0
        for batch in batch_indices:
            batch_importance = len(batch) / len(outputs)
            batch_input = inputs[batch]
            batch_output = outputs[batch]
            with torch.no_grad():
                ### Compute prediction and loss###
                batch_predictions = self.model(batch_input)
                loss = nn.CrossEntropyLoss()(batch_predictions, batch_output)
            pred = torch.argmax(batch_predictions, axis=-1)  
            batch_acc = torch.sum(pred == batch_output) / len(batch_predictions)
            losses += loss.detach().item() * batch_importance
            acc += batch_acc.detach().item() * batch_importance
        if print_acc:
            print("Accuracy: %.3f" % acc)
        return losses, acc


kf = KFold(n_splits=3, shuffle=True, random_state=42)
training_data = aev_output[1]
training_labels = Energies

training_set = []
validation_set = []
training_label = []
validation_label = []

# Will have to further study the data set to understand how to split in such a way that the various indices can be taken into account 
# for train_index, val_index in kf.split(training_data):
#     X_train, X_val = training_data[train_index], training_data[val_index]
#     y_train, y_val = training_labels[train_index], training_labels[val_index]
#     training_set.append(X_train)
#     validation_set.append(X_val)
#     training_label.append(y_train)
#     validation_label.append(y_val)

In [12]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1, padding=2)
        self.fc = nn.Sequential(
            nn.Linear(3*32*32, 10),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = x.view(-1, 1, 32, 32)
        x = F.relu(self.conv(x))
        x = x.view(-1, 3*32*32)
        x = self.fc(x)
        return x

In [13]:
# The training and validation accuracy of the model
# The training and validation accuracy of the model
run_info_list = []
for i in range(3):
    model = LeNet()
    trainer = Trainer(model, optimizer_type='adam', learning_rate=1e-3, batch_size=128, epoch=30, input_transform = lambda x:x)
    run_info = trainer.train(training_set[i], training_label[i], validation_set[i], validation_label[i])
    run_info_list.append(run_info)
    print("Training accuracy: ", run_info["accuracies"][-1])
    print("Validation accuracy: ", run_info["val_accuracies"][-1])

    # Plot the training and validation accuracy for fold 1 
    plt.plot(run_info["accuracies"], label="Training accuracy")
    plt.plot(run_info["val_accuracies"], label="Validation accuracy")
    plt.title("Training and validation accuracy for fold %d" % (i+1))
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

    # Plot the training and validation loss for fold 1
    plt.plot(run_info["losses"], label="Training loss")
    plt.plot(run_info["val_losses"], label="Validation loss")
    plt.title("Training and validation loss for fold %d" % (i+1))
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

# Summarize the training and validation accuracy of the model over the three folds 
print("Summary: ")
print("Training accuracy: ", np.mean([run_info["accuracies"][-1] for run_info in run_info_list]))
print("Validation accuracy: ", np.mean([run_info["val_accuracies"][-1] for run_info in run_info_list]))

IndexError: list index out of range